In [1]:
import twitter
import json
import os
import time
#search function is under ./Code/search.py
import sys
sys.path.append("./Code/") #add the path with the code so that it can be imported
import search


## LoadFile() opens the specified file and returns a json object of its contents


In [2]:
def LoadFile(pathName):
    if os.path.isfile(pathName):
        file = open(os.path.join(pathName), 'r')
        return json.load(file)
    else: print("File not in directory")

In [3]:
# Will sleep for n seconds

def trusty_sleep(n):
    start = time.time()
    while (time.time() - start < n):
        time.sleep(n - (time.time() - start))

## Extract

In [4]:
#Takes in a list of statuses, extracts info from the status and places into dictionart
def extract_data(statuses, mydictionary, day_number):
    '''
    Parameters:
    Statuses(list) - Iterates over a list of statuses
    mydictionary - Dictionary to be filled in
    day_number - Day number to be filled in dictionary
    
    I excluded the code for searching all the statuses for a given day, since Logan is doing this.
    All this function serves to do is extract the information and fill in the corresponding fields
    in the dictionary.
    
    '''
    day_number = 'day' + str(day_number)
    for i in statuses:
        try:
            mydictionary[day_number]['created_at'].append(i['created_at'])
            mydictionary[day_number]['followers_count'].append(i['user']['followers_count'])
            mydictionary[day_number]['retweet_count'].append(i['retweet_count'])
            # Increments the count of lang if the lang exists, else intializes it with value 1
            mydictionary[day_number]['lang'][i['user']['lang']] = mydictionary[day_number]['lang'].get(i['user']['lang'], 0) + 1
            # favorite_count is nullable, so it may not exist
            if i.get('favorite_count', False):
                mydictionary[day_number]['favorite_count'].append(i['favorite_count'])
        except (KeyError):
            continue
            
    #print(json.dumps(tempdict, indent = 1))

## Search

In [5]:
def DaySearch(htDictionary, tagIndex, startDay, endDay):
    '''
    htDictionary: The hashtags dictionary from file
    tagIndex: The "index" of the current hashtag being searched
    startDay: The starting calendar day for the search
    endDay: The ending calendar day for the search
    
    I'm calling searchOverRange function per day instead 
    of using it for multiple days to implement Sleep.'''
    
    holdResults = []
    currentHashtag = list(htDictionary.items())[tagIndex]
    for day, details in currentHashtag[1].items():
        try:
            results = search.searchOverRange(currentHashtag[0], 2018, 10, startDay, 2018, 10, startDay+1, debug=0)
        except:
            print("\nStopped at day {}".format(startDay))
            break
        if not results: break
        try:
            #print("{} requests remaining\n".format(results[-1]['metadata']['x-rate-limit-remaining']))
            numRequests = results[-1]['metadata']['x-rate-limit-remaining']
            #rint("Requests remaining: {}".format(numRequests))
        except: 
            print("\nStopped at day {}".format(startDay))
            break
            
        holdResults.append(results[0])
        startDay += 1
        if startDay >= endDay: break
        elif int(results[-1]['metadata']['x-rate-limit-remaining']) == 0: break    
    return holdResults

## Search/Extract 
`startingIndex` and `endingIndex` are used to split up the searches.

for example: `startingIndex = 100` and `endingIndex = 199` will search/extract the 100-199 hashtags in the hashtags dictionary


In [6]:
# this will hold the hashtags we extract in the same format as the hashtags dictionary

##############################################################################
##############################################################################
#
# RE-RUNNING THIS CELL WILL DELETE ALL PREVIOUS DATA SAVED IN DATADICTIONARY #
#
##############################################################################
##############################################################################

dataDictionary = {}

In [7]:
###########################
# CHANGE PARTITION NUMBER #
###########################
hashtags = LoadFile(os.path.join('Data', 'Tweets-Hashtags-Partition0.json'))


# To search in parts
startingIndex = 0
endingIndex = len(hashtags)

print("Going from %d to %d" % (startingIndex, endingIndex))

for tagIndex in range(startingIndex, endingIndex):
    tagString = list(hashtags.items())[tagIndex][0]
    dayDictionary = dict(list(hashtags.items())[tagIndex][1])
    print("\nSearching hashtag: {}\nindex {}\n".format(tagString, tagIndex))
    
    daycount = 0
    for x in range(13, 20):
        #print(tagString)
        #print(dayDictionary.keys(), end = '\n\n')
        
        resultList = DaySearch(hashtags, tagIndex, x, x+1)
        if (resultList != [[]]) and (resultList != []):
            dayDictionary['day' + str(daycount)]['occurrences'] = resultList[0]['numberOfTweets']
            extract_data(resultList[0]['tweets'], dayDictionary, daycount)
            dataDictionary[tagString] = dict(dayDictionary)
            
        if (resultList == []) or (int(resultList[-1]['metadata']['x-rate-limit-remaining']) == 0):
            print("Sleeping...")
            trusty_sleep(930)
            print("Resuming :)\n")
        
        
        daycount +=1
        
def StoreHashtags(pathName, htDictionary): 
    newPath = pathName[:-5] + "-data.json"

    with open(newPath, 'w') as f:    
        json.dump(htDictionary, f)
        f.close()
    print("Stored data in {}".format(newPath))

StoreHashtags(os.path.join('Data', 'Partition0.2.json'), dataDictionary)

Going from 0 to 1057

Searching hashtag: SDVX
index 0

Error 429: ('Too Many Requests: Request cannot be served ', "due to the application's rate limit having ", 'been exhausted for the resource')
Sleeping...
Resuming :)


Searching hashtag: ARIAsCamilaCabello
index 1


Searching hashtag: EXILETRIBE
index 2


Searching hashtag: liseli
index 3


Searching hashtag: EEUU
index 4


Searching hashtag: sarapngdaddy
index 5


Searching hashtag: LEEKNOW
index 6

Sleeping...
Resuming :)


Searching hashtag: القناص_يجلد_الشقرديه
index 7

Sleeping...
Resuming :)


Searching hashtag: ColinKaepernick
index 8


Searching hashtag: 드림캐쳐
index 9

Sleeping...
Resuming :)


Searching hashtag: 영종도출장안ㅁㅏ
index 10


Searching hashtag: غرد_لي_بالخير
index 11


Searching hashtag: チクイキ
index 12


Searching hashtag: IsraeliSettler
index 13


Searching hashtag: الأربعاء
index 14


Searching hashtag: pegging
index 15


Searching hashtag: TopQuotes
index 16


Searching hashtag: NCT2018
index 17


Searching hashtag:


Searching hashtag: 引き寄せ
index 165


Searching hashtag: スマホゲーム
index 166


Searching hashtag: 個人再生
index 167


Searching hashtag: HebeDeBonafini
index 168


Searching hashtag: 水族館
index 169


Searching hashtag: a99ii
index 170


Searching hashtag: JAEHYUN
index 171


Searching hashtag: GAZOORacing
index 172

Error 429: ('Too Many Requests: Request cannot be served ', "due to the application's rate limit having ", 'been exhausted for the resource')

Stopped at day 16
Sleeping...
Resuming :)


Searching hashtag: NYR
index 173


Searching hashtag: 艦これアーケード
index 174


Searching hashtag: 우리_지민이_생일이네
index 175


Searching hashtag: マシュマロを投げ合おう
index 176

Sleeping...
Resuming :)


Searching hashtag: 怒りと喜びと悲しみの割合
index 177


Searching hashtag: ModiRafaleVictory
index 178


Searching hashtag: BigHug4Kanye
index 179


Searching hashtag: trump
index 180


Searching hashtag: 江口のりこ
index 181


Searching hashtag: UniversalCredit
index 182

Sleeping...
Resuming :)

Sleeping...
Resuming :)


Searching

Sleeping...
Resuming :)


Searching hashtag: SeulGi
index 328

Sleeping...
Resuming :)


Searching hashtag: furrygoldendreamz
index 329


Searching hashtag: bogus
index 330


Searching hashtag: 방탄진
index 331


Searching hashtag: คังแดเนียล
index 332


Stopped at day 14
Sleeping...
Resuming :)


Searching hashtag: carnaviradio
index 333


Searching hashtag: Lullaby
index 334

Sleeping...
Resuming :)


Searching hashtag: ليه_غيرتنا_الايام
index 335


Searching hashtag: RTで私を有名にして下さい
index 336


Searching hashtag: 이삐사진전
index 337

Sleeping...
Resuming :)

Sleeping...
Resuming :)

Sleeping...
Resuming :)


Searching hashtag: 動作
index 338


Searching hashtag: 이기자페스티벌
index 339


Searching hashtag: あなたに聞いてはいけないこと
index 340


Searching hashtag: WeTheBestRadio
index 341


Searching hashtag: C94
index 342


Searching hashtag: 김동완
index 343

Sleeping...
Resuming :)


Searching hashtag: 1mmでもいいなと思ったらRT
index 344


Searching hashtag: NBTSTourPortoAlegre
index 345


Searching hashtag: 서울마사지
index 3


Searching hashtag: gochiusa
index 498


Searching hashtag: 板尾創路
index 499


Searching hashtag: 相互希望
index 500

Sleeping...
Resuming :)


Searching hashtag: 日刊イバラギン
index 501


Searching hashtag: targets
index 502


Searching hashtag: AFazenda
index 503


Searching hashtag: TAEIL
index 504


Searching hashtag: ミリシタボーダー
index 505


Searching hashtag: Alerta
index 506

Sleeping...
Resuming :)


Searching hashtag: aFazenda10
index 507


Searching hashtag: UprisingSVU
index 508


Searching hashtag: HangAsiaDefend295C
index 509


Searching hashtag: SDVX_V
index 510


Searching hashtag: cyberbacker
index 511


Searching hashtag: BookLover
index 512


Searching hashtag: AZSEN
index 513

Sleeping...
Resuming :)

Sleeping...


KeyboardInterrupt: 

## Testing

In [8]:
print("OKAy")

OKAy


In [9]:
print(dataDictionary)
print("OKAy")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(len(dataDictionary))#['トレクル']['day0']['retweet_count']))

In [ ]:
for htag, dayDict in dataDictionary.items():
    print(htag)
    for day, details in dayDict.items():
        print("  {} occurrences: {}".format(day, details['occurrences']))

In [ ]:
# The first bracket [2] is index, 
# second bracket [1] is hashtag's days dictionary, or [0] for the hashtag string

#print(json.dumps(dataDictionary, indent = 1))
print(*list(dataDictionary.items())[10][1]['day3']['lang'].items(), sep = '\n', end = '\n\n')

## Stores dataDictionary

In [11]:
def StoreHashtags(pathName, htDictionary): 
    newPath = pathName[:-5] + "-data.json"

    with open(newPath, 'w') as f:    
        json.dump(htDictionary, f)
        f.close()
    print("Stored data in {}".format(newPath))

## Change partition number in case you want to gather data at different intervals or if you reset dataDictionary to ensure you dont overwrite previously saved data

In [12]:
###########################
# CHANGE PARTITION NUMBER #
###########################

StoreHashtags(os.path.join('Data', 'Partition1.2.json'), dataDictionary)

Stored data in Data\Partition1.2-data.json
